# supplementary figure 1 - marker genes for all populations

In [ ]:
require(tidyverse)
require(ComplexHeatmap)
require(circlize)
require(data.table)
suppressPackageStartupMessages({
    require(tidyverse)
    require(sf)
    require(data.table)
    require(Matrix)
    require(ggpubr)
    require(ggthemes)
    require(future)
    require(furrr)
    require(lme4)
    require(presto)
    require(ggrepel)
    require(Seurat)
    require(ComplexHeatmap)
    require(circlize)
    sf::sf_use_s2(FALSE)  
    set.seed(1)
})

## load meta data

In [ ]:
metadata = readr::read_rds('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH_niches/labeled_seurat_objects/renamed_cell_states/metadata_complete.rds')
metadata$KNN_group = as.vector(metadata$KNN_group)
metadata$knn_renamed_cell_states = as.vector(metadata$knn_renamed_cell_states)
metadata = metadata %>% 
    mutate(KNN_group = ifelse(knn_coarse == 'B', knn_renamed_cell_states, KNN_group)) %>% 
    mutate(knn_renamed_cell_states = ifelse(knn_renamed_cell_states == 'Fibro-CXCL14', 'Fibro-GREM1-low', knn_renamed_cell_states)) 
sample_n(metadata, 10)

## load pathology regions

In [ ]:
pathology_regions = data.table::fread('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH/compare_CXCL10_niches_and_tessera_20240930/adata_all_obs.csv') %>%
    select(!V1) %>%
    mutate(orig_cell_id = sample_cell) 

# remove 'submucosa' regions

pathology_regions$pathology_region[pathology_regions$pathology_region == 'non_neoplastic_submucosa'] = 'Unannotated'
pathology_regions$sample_name[pathology_regions$sample_name == 'G4659'] = 'G4659-CP-MET_VMSC04701'
pathology_regions$orig_cell_id[pathology_regions$sample_name == 'G4659-CP-MET_VMSC04701'] = paste0('G4659-CP-MET_VMSC04701_', pathology_regions$cell_id[pathology_regions$sample_name == 'G4659-CP-MET_VMSC04701'])

pathology_regions$sample_name[pathology_regions$sample_name == 'G4659_Beta8'] = 'G4659-CP-MET_Beta8'
pathology_regions$orig_cell_id[pathology_regions$sample_name == 'G4659-CP-MET_Beta8'] = paste0('G4659-CP-MET_Beta8_', pathology_regions$cell_id[pathology_regions$sample_name == 'G4659-CP-MET_Beta8'])

pathology_regions$sample_cell = paste0(pathology_regions$sample_name, '_', pathology_regions$cell_id)
pathology_regions$orig_cell_id = paste0(pathology_regions$sample_name, '_', pathology_regions$cell_id)

In [ ]:
pathology_regions$MSstatus[pathology_regions$sample_name == 'C107'] %>% unique

In [ ]:
pathology_regions = pathology_regions %>%
    mutate(condition = MSstatus,
          knn_renamed_cell_states = KNN_celltype_v2,
          orig.ident = sample_name,
          knn_coarse = KNN_Top,
          #pathology_region = ifelse(grepl(pathology_region, pattern = 'tumor'), yes = 'tumor', no = pathology_region),
          ) %>%
    mutate(knn_coarse = ifelse(knn_coarse == 'Mast', yes = 'Myeloid', no = ifelse(knn_coarse %in% c('B', 'Plasma'), yes = 'Bplasma', no = knn_coarse))) %>%
    mutate(knn_renamed_cell_states = ifelse(knn_coarse == 'Epi', yes = 'Epi', no = knn_renamed_cell_states)) %>%
    mutate(knn_renamed_cell_states = ifelse(knn_renamed_cell_states == 'Fibro-CXCL14', 'Fibro-GREM1-low', knn_renamed_cell_states)) %>% 
    mutate(pathology_region = gsub(pathology_region, pattern = '_', replacement = ' ')) %>%
    mutate(pathology_region = str_to_title(pathology_region)) %>%
    mutate(pathology_region = gsub(pathology_region, pattern = 'Muscularis Propria', replacement = 'M. propria')) %>%
    mutate(pathology_region = gsub(pathology_region, pattern = 'Non Neoplastic Mucosa', replacement = 'Non-neo. muc.')) %>%
    mutate(pathology_region = gsub(pathology_region, pattern = 'Non Neoplastic Submucosa|non_neoplastic_submucosa', replacement = 'Unannotated')) %>%  #'Non-neo. submuc.'
    mutate(pathology_region = gsub(pathology_region, pattern = 'Tumor Invasive Margin', replacement = 'Tumor inv-border')) %>%
    mutate(pathology_region = gsub(pathology_region, pattern = 'Tumor Luminal Margin', replacement = 'Tumor lum-border'))

In [ ]:
sample_n(pathology_regions, 10)

## frequencies of region per sample show that some regions are extremely small and must be removed

In [ ]:
.x = metadata %>% 
    left_join(., pathology_regions %>% select(sample_cell, pathology_region, PatientID) %>% rename(cell = sample_cell)) %>%
    mutate(pathology_region = gsub(pathology_region, pattern = '_', replacement = ' '))
table(.x$pathology_region, .x$orig.ident) %>% as.data.frame %>% filter(Freq > 0)

In [ ]:
table(.x$pathology_region, .x$orig.ident) %>% as.data.frame %>% filter(Freq > 0) %>% 
pivot_wider(names_from = Var1, values_from = Freq, values_fill = 0)

In [ ]:
table(.x$pathology_region, .x$orig.ident) %>% as.data.frame %>% filter(Freq < 250 & Freq > 0)

In [ ]:
table(.x$pathology_region, .x$PatientID) %>% as.data.frame %>% filter(Freq > 250) %>% 
pivot_wider(names_from = Var1, values_from = Freq, values_fill = 0) %>%
tibble::column_to_rownames('Var2')

## QC regions < 250 cells

In [ ]:
table(.x$pathology_region, 
      .x$PatientID) %>% 
    as.data.frame %>% 
    filter(Freq > 250) %>%
    select(Var1, Var2) %>%
    rename(pathology_region = Var1, PatientID = Var2) %>%
    mutate(pathology_region_PatientID = paste0(pathology_region, '_', PatientID))

selected_pathology_regions = table(.x$pathology_region, 
      .x$PatientID) %>% 
    as.data.frame %>% 
    filter(Freq > 250) %>%
    select(Var1, Var2) %>%
    rename(pathology_region = Var1, PatientID = Var2) %>%
    mutate(pathology_region_PatientID = paste0(pathology_region, '_', PatientID)) %>%
    pull(pathology_region_PatientID)

In [ ]:
pathology_regions %>% dim
pathology_regions_preQC = pathology_regions
pathology_regions = pathology_regions %>% 
    mutate(pathology_region_PatientID = paste0(pathology_region, '_', PatientID)) %>%
    filter(pathology_region_PatientID %in% selected_pathology_regions) 
pathology_regions %>%
    dim

In [ ]:
nCells_pathology = pathology_regions %>% 
    left_join(., metadata %>% select(cell_id, orig.ident, area)) %>%
    mutate(orig.ident = gsub(orig.ident, pattern = '_.*', replacement = '')) %>%
    select(condition, knn_renamed_cell_states, orig.ident, KNN_Top, pathology_region, area) %>%
    group_by(orig.ident, knn_renamed_cell_states, condition, KNN_Top, pathology_region) %>%
    summarize(total_cell_state = n(), cell_state_area = sum(area), .groups = 'keep') %>%
    ungroup %>%
    group_by(orig.ident, condition, pathology_region, .drop = FALSE) %>%
    mutate(total_region = sum(total_cell_state), total_region_area = sum(cell_state_area)) %>%
    ungroup %>%
    group_by(orig.ident, .drop = FALSE) %>%
    mutate(total_cells = sum(total_cell_state), total_area = sum(total_region_area)) %>%
    mutate(cell_state_as_percent_of_total = 100*total_cell_state/total_cells, cell_state_as_percent_of_region = 100*total_cell_state/total_region, cell_state_density_area = cell_state_area/total_region_area) %>%
    ungroup 

nCells_pathology %>%
    sample_n(20)

In [ ]:
dim(nCells_pathology)

In [ ]:
unique(metadata$knn_renamed_cell_states)
unique(pathology_regions$knn_renamed_cell_states)

### counts

In [ ]:
system.time({counts = readr::read_rds('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH_niches/labeled_seurat_objects/renamed_cell_states/counts_complete.rds')}) # 50 s
require(Matrix)
system.time({counts = Matrix(as.matrix(counts), sparse = TRUE)})
dim(counts)

In [ ]:
gene_panel = read.table('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH/figure6/merfish_gene_panel.tsv')$x
gene_panel %>% length

In [ ]:
rownames(counts) = gene_panel

In [ ]:
table(metadata$knn_renamed_cell_states[metadata$knn_coarse == 'Strom'])

In [ ]:
unique(metadata$knn_coarse)
unique(metadata$knn_mid_renamed)
unique(pathology_regions$KNN_Mid)

# expression in B cells

# Cell States

In [ ]:
require(furrr)
require(future)
require(presto)
plan(multisession)
require(singlecellmethods)

effects_marginal_cell_states = lapply(c('Strom', 'Epi', 'Myeloid', 'TNKILC', 'B'), function(lineage){
    .metadata = metadata %>%
        na.omit() %>%
        filter(knn_coarse == lineage) %>%
        select(orig.ident, condition, cell, knn_renamed_cell_states) %>%
        distinct %>%
        mutate(orig.ident = gsub(orig.ident, pattern = '_.*', replacement = "")) %>%
        distinct %>%
        as.data.frame

    rownames(.metadata) = .metadata$cell
    .counts = counts[, rownames(.metadata)]

    .metadata$logUMI = log(colSums(.counts) + 1)

    pb = presto::collapse_counts(
        counts_mat = .counts, 
        meta_data = .metadata,
        c('orig.ident', 'knn_renamed_cell_states'), 
        min_cells_per_group = 3
    )
    
    system.time({
    presto_res = presto::presto.presto(
        formula = y ~ 1 + (1|knn_renamed_cell_states) +  (1|orig.ident/knn_renamed_cell_states) + offset(logUMI),
        #formula = y ~ 1 + (1|pathology_region) + (1|orig.ident) + offset(logUMI),  # 
        #formula = y ~ 1 + (1|pathology_region) + (1|orig.ident/pathology_region) + offset(logUMI), 
        design = pb$meta_data, #.metadata, 
        response = pb$counts_mat, #.counts,
        size_varname = "logUMI", 
        effects_cov = 'knn_renamed_cell_states',
        ncore = 10, 
        min_sigma = .05,
        family = "poisson",
        nsim = 1000 
    )}) # failed with error Detected a non-exportable reference (‘externalptr’) in one of the globals (<unknown>) used in the future expression. 

    contrasts_mat = make_contrast.presto(
        presto_res, 
        var_contrast = 'knn_renamed_cell_states')

    effects_marginal = contrasts.presto(
        presto_res, 
        contrasts_mat, 
        one_tailed = FALSE
    ) %>% 
        dplyr::mutate(
            logFC = sign(beta) * log2(exp(abs(beta))), # convert stats to log2 for interpretability 
            SD = log2(exp(sigma)),
            zscore = logFC / SD,
            fdr = p.adjust(pvalue, method = 'fdr')
        ) %>%
        arrange(pvalue) %>%
        mutate(knn_coarse = lineage)
    return(effects_marginal)
})

In [ ]:
effects_marginal_cell_states = rbindlist(effects_marginal_cell_states)

In [ ]:
slice_sample(effects_marginal_cell_states, n = 20)

In [ ]:
fwrite('supplementary_table_1_fig_1.csv', x = effects_marginal_cell_states)

In [ ]:
require(tidyverse)
data.table::fread('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH/CRC_Figure_1/supplementary_table_1_fig_1.csv') %>%
    filter(feature == 'CXCR3' & knn_coarse == 'TNKILC') %>% 
    arrange(desc(logFC))
#& grepl(contrast, pattern = 'Tcd8')) 

# make heatmaps

In [ ]:
makeMarkerHeatmap = function(lineage, effects_marginal_cell_states, canonicalMarkers = c()){
    .effects = effects_marginal_cell_states %>% filter(knn_coarse == lineage)
    if (length(canonicalMarkers) == 0){
        canonicalMarkers = .effects %>%
                filter(fdr < 0.05 & logFC > 1) %>%
                group_by(contrast) %>%
                slice_max(n = 15, order_by = logFC) %>%
                pull(feature) %>%
                unique
        # if (length(canonicalMarkers) > 100){
        #     canonicalMarkers = .effects %>%
        #         filter(fdr < 0.05 & logFC > 1) %>%
        #         group_by(contrast) %>%
        #         slice_max(n = 5, order_by = logFC) %>%
        #         pull(feature) %>%
        #         unique
        # }  
        # message(length(canonicalMarkers))
    }else{canonicalMarkers = canonicalMarkers}

    .temp = .effects %>%
        filter(feature %in% canonicalMarkers) %>%
        select(contrast, feature, logFC) %>%
        pivot_wider(names_from = feature, values_from = logFC) %>%
        column_to_rownames('contrast') %>%
        as.matrix
    
    #print(.temp)

    .rowAnno = metadata %>%
        filter(knn_coarse == lineage) %>%
        select(cell, knn_renamed_cell_states) %>%
        group_by(knn_renamed_cell_states) %>%
        summarize(n = n()) %>%
        ungroup %>%
        distinct %>%
        na.omit %>%
        as.data.frame
    rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
    #print(.rowAnno)
    .pval = .effects %>%
        filter(feature %in% canonicalMarkers) %>%
        select(contrast, feature, fdr, logFC) %>%
        mutate(fdr = ifelse(fdr < 0.05 & logFC > 1, yes = '*', no = '')) %>%
        select(!logFC) %>%
        pivot_wider(names_from = feature, values_from = fdr) %>%
        column_to_rownames('contrast') %>%
        as.matrix
    .temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
    .pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

    options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
    set.seed(1)
    ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
        `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
        annotation_name_rot = 0,
        which = 'row'
        )
    #col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c(scales::muted('blue'), "white", scales::muted('red')))
    col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
    tnkilc = ComplexHeatmap::Heatmap(
                            row_title = 'Cell States', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Marker Genes',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = TRUE,
                            cluster_columns = TRUE,
                            right_annotation = ha2,
                            #cell_fun = function(j, i, x, y, width, height, fill) {grid.text(.pval[i, j], x, y, gp = gpar(fontcolor = 'red', fontsize = 10))},
                            name = 'logFC',
                            column_names_side = 'top',
                            show_column_dend = FALSE,
                            show_row_dend = FALSE,
                            matrix = .temp,
                            row_names_side = 'left')
    # draw(tnkilc,
    #      merge_legend = TRUE, 
    #      heatmap_legend_side = "right", 
    #      annotation_legend_side = "right")    
    # degs_heatmap = grid::grid.grabExpr(draw(tnkilc,
    #  merge_legend = TRUE, 
    #  heatmap_legend_side = "right", 
    #  annotation_legend_side = "right"))
    return(tnkilc)
}


In [ ]:
effects_marginal_cell_states %>% filter(knn_coarse == 'Strom') %>% pull(contrast) %>% unique %>% writeLines

In [ ]:
markerHeatmaps = lapply(c('Strom', 'Epi', 'Myeloid', 'TNKILC', 'B'), makeMarkerHeatmap, effects_marginal_cell_states = effects_marginal_cell_states)

### Strom DEGs

In [ ]:
geneList_Strom = str_split(pattern = '\n', 'MFAP5
SFRP2
CXCL12
CXCL14
CXCR4
BMP4
ENHO
BMP5
BMP2
WNT2B
CCL2
GREM1
INHBA
FAP
PDPN
MMP1
MMP3
CXCL3
CXCL5
CXCL8
CSF3
IL24
RSPO3
TAGLN
ACTA2
MYH11
TNFSF15
CSF1
IL6
NOTCH3
CSPG4
ENHO
TSLP
UBE2C
TOP2A
MKI67
SELP
ACKR1
SELE
NCAM1
SPP1
NRXN1
TCF7
ACTG2
MYH11
GJA4
GJA5
SEMA3G
ESM1
FLT1
CD34
KDR
DLL4
JAG2
VWF
NOTCH4
PDGFB
CD36
PGF
CCL21
PROX1
ACKR2
LYVE1
FLT4') %>% unlist

In [ ]:
options(repr.plot.res = 500, repr.plot.width = 30, repr.plot.height = 6)
stromDEGs = makeMarkerHeatmap(lineage = 'Strom', canonicalMarkers = unique(intersect(geneList, effects_marginal_cell_states$feature)) , effects_marginal_cell_states = effects_marginal_cell_states)
draw(stromDEGs)

## TNKILC

In [ ]:
geneList_TNKILC = str_split(pattern = '\n', 
'TCF7
CXCR3
IL7R
CCR7
SELL
LEF1
BCL6
CXCR5
TIGIT
ZBED2
CXCL13
RBPJ
TOX
TNFSF4
IL2RA
FOXP3
BATF
CTLA4
TNFRSF4
STMN1
MK167
HMGRB2
KLRG1
GZMK
CD8A
CD8B
ZNF683
HSP1A
ITGAE
CCL5
ENTPD1
PDCD1
HAVR2
IFNG
LAG3
TRDC1
TRGC1
SGNLY
PRF1
XCXL1
CMC1
LST1
RORC')
require(tidyverse)
effects_marginal_cell_states = data.table::fread('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH/CRC_Figure_1/supplementary_table_1_fig_1.csv') 


In [ ]:
pdf(file = 'tnkilc_markers.pdf', width = 35, height = 6)
makeMarkerHeatmap(lineage = 'TNKILC', canonicalMarkers = unique(intersect(geneList_TNKILC, effects_marginal_cell_states$feature)) , effects_marginal_cell_states = effects_marginal_cell_states)
dev.off()

In [ ]:
options(repr.plot.res = 300, repr.plot.width = 35, repr.plot.height = 6)
dev.off()
tDEGs = makeMarkerHeatmap(lineage = 'TNKILC', canonicalMarkers = unique(intersect(geneList_TNKILC, effects_marginal_cell_states$feature)) , effects_marginal_cell_states = effects_marginal_cell_states)
draw(tDEGs)

In [ ]:
unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CCR|CXCR')])

In [ ]:
require(ComplexHeatmap)
effects_marginal_cell_states = fread('supplementary_table_1_fig_1.csv')
geneList_Myeloid = unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CXCL')]), 
                                                              effects_marginal_cell_states$feature))
canonicalMarkers =  unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CXCL')]), 
                                                              effects_marginal_cell_states$feature))
.effects = effects_marginal_cell_states %>% 
filter(knn_coarse == 'TNKILC') 
.temp = .effects %>%
filter(feature %in% geneList_Myeloid) %>%
select(contrast, feature, logFC) %>%
pivot_wider(names_from = feature, values_from = logFC) %>%
column_to_rownames('contrast') %>%
as.matrix
lineage = 'TNKILC'
.rowAnno = metadata %>%
    filter(knn_coarse == lineage) %>%
    select(cell, knn_renamed_cell_states) %>%
    group_by(knn_renamed_cell_states) %>%
    summarize(n = n()) %>%
    ungroup %>%
    distinct %>%
    na.omit %>%
    as.data.frame
rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
#print(.rowAnno)
.pval = .effects %>%
    filter(feature %in% canonicalMarkers) %>%
    select(contrast, feature, fdr, logFC) %>%
    mutate(fdr = ifelse(fdr < 0.05 & logFC > 0.5, yes = '*', no = '')) %>%
    select(!logFC) %>%
    pivot_wider(names_from = feature, values_from = fdr) %>%
    column_to_rownames('contrast') %>%
    as.matrix
.temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
.pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
set.seed(1)
ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
    `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
    annotation_name_rot = 0,
    which = 'row'
    )
range(.temp)
col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
ligand = ComplexHeatmap::Heatmap(.temp,
                             row_title = 'Cell States', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Ligand',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = TRUE,
                            cluster_columns = TRUE,
                            #right_annotation = ha2,
        cell_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'Ligand',
                            column_names_side = 'top',
                            show_column_dend = TRUE,
                            show_row_dend = TRUE,
                            row_names_side = 'left')

       
ligand2 = draw(ligand)
row_order = row_order(ligand2)

geneList_Myeloid = unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCR|CXCR')]), 
                                                              effects_marginal_cell_states$feature))
canonicalMarkers =  unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCR|CXCR')]), 
                                                              effects_marginal_cell_states$feature))
.effects = effects_marginal_cell_states %>% 
filter(knn_coarse == 'TNKILC') 
.temp = .effects %>%
filter(feature %in% geneList_Myeloid) %>%
select(contrast, feature, logFC) %>%
pivot_wider(names_from = feature, values_from = logFC) %>%
column_to_rownames('contrast') %>%
as.matrix
lineage = 'TNKILC'
.rowAnno = metadata %>%
    filter(knn_coarse == lineage) %>%
    select(cell, knn_renamed_cell_states) %>%
    group_by(knn_renamed_cell_states) %>%
    summarize(n = n()) %>%
    ungroup %>%
    distinct %>%
    na.omit %>%
    as.data.frame
rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
#print(.rowAnno)
.pval = .effects %>%
    filter(feature %in% canonicalMarkers) %>%
    select(contrast, feature, fdr, logFC) %>%
    mutate(fdr = ifelse(fdr < 0.05 & logFC > 0.5, yes = '*', no = '')) %>%
    select(!logFC) %>%
    pivot_wider(names_from = feature, values_from = fdr) %>%
    column_to_rownames('contrast') %>%
    as.matrix
.temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
.pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
set.seed(1)
ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
    `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
    annotation_name_rot = 0,
    which = 'row'
    )
col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
receptor = ComplexHeatmap::Heatmap(.temp,
                             row_title = 'Cell State', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Receptor',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = FALSE,
                            cluster_columns = TRUE,
                            row_order = row_order,
                            #right_annotation = ha2,
        cell_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'Receptor',
                            column_names_side = 'top',
                            show_column_dend = TRUE,
                            show_row_dend = TRUE,
                            row_names_side = 'left')



In [ ]:
pdf('TNKILC_chemokines.pdf', width = 30, height = 6)
ligand + receptor
dev.off()

# B

In [ ]:
geneList_B = str_split(pattern = '\n', 'MZB1
CD27
CD38
CD79A
MS4A1
CD19
CXXCR5
CD74
TNFRSF13B
GPR183
CD40
STMN1
HMGB2
MK167')

In [ ]:
options(repr.plot.res = 500, repr.plot.width = 30, repr.plot.height = 6)
bDEGs = makeMarkerHeatmap(lineage = 'B', canonicalMarkers = unique(intersect(geneList_B, effects_marginal_cell_states$feature)) , effects_marginal_cell_states = effects_marginal_cell_states)
draw(bDEGs)

In [ ]:
effects_marginal_cell_states = fread('supplementary_table_1_fig_1.csv')
geneList_Myeloid = unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CCR|CXCR|CXCL')]), 
                                                              effects_marginal_cell_states$feature))
canonicalMarkers =  unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CCR|CXCR|CXCL')]), 
                                                              effects_marginal_cell_states$feature))
.effects = effects_marginal_cell_states %>% 
filter(knn_coarse == 'B') 
.temp = .effects %>%
filter(feature %in% geneList_Myeloid) %>%
select(contrast, feature, logFC) %>%
pivot_wider(names_from = feature, values_from = logFC) %>%
column_to_rownames('contrast') %>%
as.matrix
lineage = 'B'
.rowAnno = metadata %>%
    filter(knn_coarse == lineage) %>%
    select(cell, knn_renamed_cell_states) %>%
    group_by(knn_renamed_cell_states) %>%
    summarize(n = n()) %>%
    ungroup %>%
    distinct %>%
    na.omit %>%
    as.data.frame
rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
#print(.rowAnno)
.pval = .effects %>%
    filter(feature %in% canonicalMarkers) %>%
    select(contrast, feature, fdr, logFC) %>%
    mutate(fdr = ifelse(fdr < 0.05 & logFC > 0.5, yes = '*', no = '')) %>%
    select(!logFC) %>%
    pivot_wider(names_from = feature, values_from = fdr) %>%
    column_to_rownames('contrast') %>%
    as.matrix
.temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
.pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
set.seed(1)
ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
    `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
    annotation_name_rot = 0,
    which = 'row'
    )
range(.temp)
col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
   tnkilc = ComplexHeatmap::Heatmap(.temp,
                             row_title = 'Cell States', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Marker Genes',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = TRUE,
                            cluster_columns = TRUE,
                            #right_annotation = ha2,
        cell_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'logFC',
                            column_names_side = 'top',
                            show_column_dend = FALSE,
                            show_row_dend = FALSE,
                            row_names_side = 'left')

       
tnkilc

pdf('B_chemokines.pdf', width = 30, height = 6)
tnkilc
dev.off()

# Epi

In [ ]:
geneList_Epi = str_split(pattern = '\n', 'LGR5
LEFTY1
ADH1B1
EPHB3
CDCA7
STMN1
MK167
HMGB2
SPINK1
OLFM4
ITLN1
CLCA1
MUC2
CHGA
IL3RA
CLCA4
AQP8
TMGD1
SULT1A2
SLC26A3
SLC26A2
CA1
GUCA2B
BEST4
AZGP1
HCK
HPGDS
PTGS1')

In [ ]:
options(repr.plot.res = 500, repr.plot.width = 35, repr.plot.height = 6)
epiDEGs = makeMarkerHeatmap(lineage = 'Epi', canonicalMarkers = unique(intersect(geneList_Epi, effects_marginal_cell_states$feature)) , effects_marginal_cell_states = effects_marginal_cell_states)
draw(epiDEGs)

# Myeloid

In [ ]:
geneList_Myeloid = str_split(pattern = '\n', 'CXCR3
CXCL9
CXCL10
CXCL11
CLEC4C
LILRA4
CLEC9A
XCR1
CCL19
CCL21
LAMP3
CCR7
CD1E
CD1C
FCER1A
C1QA
CD163
CXCL16
HIF1A
SEPP1
LYVE1
FOLR2
MMP9
MMP2CCL18
APOE
C1QA
C1QC
C1QB
AXL
STMN1
MK167
HMGB2
ISG15
STAT1
CXLC9
CXCL10
CD274
GBP1
TAP1
CSF1R
LYZ
S100A8
VCAN
FCN1
CSF3R
CCL3
CCL20
CXCL3
CXCL2
TNF
IL6IL1B
IL1A
NLRP3
VEGFA
EREG
FCGR3B
HCAR2
PTGS2
CXCL8') %>% unlist
geneList_Myeloid

In [ ]:
sum(geneList_Myeloid %in% .effects$feature) 
length(geneList_Myeloid)

In [ ]:
head(.effects)

In [ ]:
canonicalMarkers = intersect(geneList_Myeloid, effects_marginal_cell_states$feature)
.effects = effects_marginal_cell_states %>% 
filter(knn_coarse == 'Myeloid') 
.temp = .effects %>%
filter(feature %in% geneList_Myeloid) %>%
select(contrast, feature, logFC) %>%
pivot_wider(names_from = feature, values_from = logFC) %>%
column_to_rownames('contrast') %>%
as.matrix
lineage = 'Myeloid'
.rowAnno = metadata %>%
    filter(knn_coarse == lineage) %>%
    select(cell, knn_renamed_cell_states) %>%
    group_by(knn_renamed_cell_states) %>%
    summarize(n = n()) %>%
    ungroup %>%
    distinct %>%
    na.omit %>%
    as.data.frame
rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
#print(.rowAnno)
.pval = .effects %>%
    filter(feature %in% canonicalMarkers) %>%
    select(contrast, feature, fdr, logFC) %>%
    mutate(fdr = ifelse(fdr < 0.05 & logFC > 0.5, yes = '*', no = '')) %>%
    select(!logFC) %>%
    pivot_wider(names_from = feature, values_from = fdr) %>%
    column_to_rownames('contrast') %>%
    as.matrix
.temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
.pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
set.seed(1)
ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
    `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
    annotation_name_rot = 0,
    which = 'row'
    )
range(.temp)
col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
   tnkilc = ComplexHeatmap::Heatmap(.temp,
                             row_title = 'Cell States', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Marker Genes',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = TRUE,
                            cluster_columns = TRUE,
                            #right_annotation = ha2,
        cell_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'logFC',
                            column_names_side = 'top',
                            show_column_dend = FALSE,
                            show_row_dend = FALSE,
                            row_names_side = 'left')
       
tnkilc

In [ ]:
require(ComplexHeatmap)
effects_marginal_cell_states = fread('supplementary_table_1_fig_1.csv')
geneList_Myeloid = unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CXCL')]), 
                                                              effects_marginal_cell_states$feature))
canonicalMarkers =  unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCL|CXCL')]), 
                                                              effects_marginal_cell_states$feature))
.effects = effects_marginal_cell_states %>% 
filter(knn_coarse == 'Myeloid') 
.temp = .effects %>%
filter(feature %in% geneList_Myeloid) %>%
select(contrast, feature, logFC) %>%
pivot_wider(names_from = feature, values_from = logFC) %>%
column_to_rownames('contrast') %>%
as.matrix
lineage = 'Myeloid'
.rowAnno = metadata %>%
    filter(knn_coarse == lineage) %>%
    select(cell, knn_renamed_cell_states) %>%
    group_by(knn_renamed_cell_states) %>%
    summarize(n = n()) %>%
    ungroup %>%
    distinct %>%
    na.omit %>%
    as.data.frame
rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
#print(.rowAnno)
.pval = .effects %>%
    filter(feature %in% canonicalMarkers) %>%
    select(contrast, feature, fdr, logFC) %>%
    mutate(fdr = ifelse(fdr < 0.05 & logFC > 0.5, yes = '*', no = '')) %>%
    select(!logFC) %>%
    pivot_wider(names_from = feature, values_from = fdr) %>%
    column_to_rownames('contrast') %>%
    as.matrix
.temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
.pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
set.seed(1)
ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
    `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
    annotation_name_rot = 0,
    which = 'row'
    )
range(.temp)
col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
ligand = ComplexHeatmap::Heatmap(.temp,
                             row_title = 'Cell States', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Ligand',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = TRUE,
                            cluster_columns = TRUE,
                            #right_annotation = ha2,
        cell_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'Ligand',
                            column_names_side = 'top',
                            show_column_dend = TRUE,
                            show_row_dend = TRUE,
                            row_names_side = 'left')

       
ligand2 = draw(ligand)
row_order = row_order(ligand2)

geneList_Myeloid = unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCR|CXCR')]), 
                                                              effects_marginal_cell_states$feature))
canonicalMarkers =  unique(intersect(unique(gene_panel[grepl(gene_panel, pattern = 'CCR|CXCR')]), 
                                                              effects_marginal_cell_states$feature))
.effects = effects_marginal_cell_states %>% 
filter(knn_coarse == 'Myeloid') 
.temp = .effects %>%
filter(feature %in% geneList_Myeloid) %>%
select(contrast, feature, logFC) %>%
pivot_wider(names_from = feature, values_from = logFC) %>%
column_to_rownames('contrast') %>%
as.matrix
lineage = 'Myeloid'
.rowAnno = metadata %>%
    filter(knn_coarse == lineage) %>%
    select(cell, knn_renamed_cell_states) %>%
    group_by(knn_renamed_cell_states) %>%
    summarize(n = n()) %>%
    ungroup %>%
    distinct %>%
    na.omit %>%
    as.data.frame
rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
#print(.rowAnno)
.pval = .effects %>%
    filter(feature %in% canonicalMarkers) %>%
    select(contrast, feature, fdr, logFC) %>%
    mutate(fdr = ifelse(fdr < 0.05 & logFC > 0.5, yes = '*', no = '')) %>%
    select(!logFC) %>%
    pivot_wider(names_from = feature, values_from = fdr) %>%
    column_to_rownames('contrast') %>%
    as.matrix
.temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
.pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
set.seed(1)
ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
    `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
    annotation_name_rot = 0,
    which = 'row'
    )
col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
receptor = ComplexHeatmap::Heatmap(.temp,
                             row_title = 'Cell State', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Receptor',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = FALSE,
                            cluster_columns = TRUE,
                            row_order = row_order,
                            #right_annotation = ha2,
        cell_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'Receptor',
                            column_names_side = 'top',
                            show_column_dend = TRUE,
                            show_row_dend = TRUE,
                            row_names_side = 'left')



In [ ]:
ligand + receptor

In [ ]:
pdf('Myeloid_chemokines.pdf', width = 30, height = 6)
ligand + receptor
dev.off()

In [ ]:
options(repr.plot.res = 500, repr.plot.width = 25, repr.plot.height = 6)
getwd()
makeMarkerHeatmap = function(lineage, effects_marginal_cell_states, canonicalMarkers = c()){
    .effects = effects_marginal_cell_states %>% filter(knn_coarse == lineage)
    if (length(canonicalMarkers) == 0){
        canonicalMarkers = .effects %>%
                filter(fdr < 0.05 & logFC > 1) %>%
                group_by(contrast) %>%
                slice_max(n = 15, order_by = logFC) %>%
                pull(feature) %>%
                unique
        # if (length(canonicalMarkers) > 100){
        #     canonicalMarkers = .effects %>%
        #         filter(fdr < 0.05 & logFC > 1) %>%
        #         group_by(contrast) %>%
        #         slice_max(n = 5, order_by = logFC) %>%
        #         pull(feature) %>%
        #         unique
        # }  
        # message(length(canonicalMarkers))
    }else{canonicalMarkers = canonicalMarkers}

    .temp = .effects %>%
        filter(feature %in% canonicalMarkers) %>%
        select(contrast, feature, logFC) %>%
        pivot_wider(names_from = feature, values_from = logFC) %>%
        column_to_rownames('contrast') %>%
        as.matrix
    
    #print(.temp)

    .rowAnno = metadata %>%
        filter(knn_coarse == lineage) %>%
        select(cell, knn_renamed_cell_states) %>%
        group_by(knn_renamed_cell_states) %>%
        summarize(n = n()) %>%
        ungroup %>%
        distinct %>%
        na.omit %>%
        as.data.frame
    rownames(.rowAnno) = .rowAnno$knn_renamed_cell_states
    #print(.rowAnno)
    .pval = .effects %>%
        filter(feature %in% canonicalMarkers) %>%
        select(contrast, feature, fdr, logFC) %>%
        mutate(fdr = ifelse(fdr < 0.05 & logFC > 1, yes = '*', no = '')) %>%
        select(!logFC) %>%
        pivot_wider(names_from = feature, values_from = fdr) %>%
        column_to_rownames('contrast') %>%
        as.matrix
    .temp = .temp[.rowAnno$knn_renamed_cell_states, canonicalMarkers]
    .pval = .pval[.rowAnno$knn_renamed_cell_states, canonicalMarkers]

    options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 5)
    set.seed(1)
    ha2 = HeatmapAnnotation(annotation_name_offset = unit(1, 'cm'),
        `log10(Cell Count)` = anno_barplot(log10(.rowAnno$n)),
        annotation_name_rot = 0,
        which = 'row'
        )
    #col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c(scales::muted('blue'), "white", scales::muted('red')))
    col_fun = colorRamp2(c(min(.temp), 0, max(.temp)), c('white', 'white', scales::muted('navyblue')))
    tnkilc = ComplexHeatmap::Heatmap(
                            row_title = 'Cell States', 
                            row_title_side = 'left',
                            column_title_side = 'bottom',
                            column_title = 'Marker Genes',
                            heatmap_legend_param = list(direction = 'vertical'),
                            col = col_fun,
                            cluster_rows = TRUE,
                            cluster_columns = TRUE,
                            right_annotation = ha2,
        layer_fun = function(j, i, x, y, width, height, fill) {
            # This uses the same logic but is vectorized and more stable
            grid.text(
                .pval[i, j], 
                x, 
                y, 
                gp = gpar(col = 'red', fontsize = 10)
            )
        },
                            name = 'logFC',
                            column_names_side = 'top',
                            show_column_dend = FALSE,
                            show_row_dend = FALSE,
                            matrix = .temp,
                            row_names_side = 'left')
    # draw(tnkilc,
    #      merge_legend = TRUE, 
    #      heatmap_legend_side = "right", 
    #      annotation_legend_side = "right")    
    # degs_heatmap = grid::grid.grabExpr(draw(tnkilc,
    #  merge_legend = TRUE, 
    #  heatmap_legend_side = "right", 
    #  annotation_legend_side = "right"))
    return(tnkilc)
}
dev.off()
effects_marginal_cell_states = fread('supplementary_table_1_fig_1.csv')
myeloidDEGs = makeMarkerHeatmap(lineage = 'Myeloid', canonicalMarkers = unique(intersect(geneList_Myeloid, effects_marginal_cell_states$feature)) , effects_marginal_cell_states = effects_marginal_cell_states)
draw(myeloidDEGs)

# Assemble Figure

In [ ]:
require(patchwork)
require(grid)

In [ ]:
options(repr.plot.res = 200, repr.plot.width = 30, repr.plot.height = 30)
supp_fig_1 = (wrap_elements(grid.grabExpr(draw(bDEGs), wrap = TRUE, wrap.grobs = TRUE)) + ggtitle('B') + theme(title = element_text(size = 30))) +
    (wrap_elements(grid.grabExpr(draw(tDEGs), wrap = TRUE, wrap.grobs = TRUE))+ ggtitle('TNKILC') + theme(title = element_text(size = 30))) +
    (wrap_elements(grid.grabExpr(draw(myeloidDEGs), wrap = TRUE, wrap.grobs = TRUE)) + ggtitle('Myeloid') + theme(title = element_text(size = 30))) +
    (wrap_elements(grid.grabExpr(draw(stromDEGs), wrap = TRUE, wrap.grobs = TRUE)) + ggtitle('Strom') + theme(title = element_text(size = 30))) +
    (wrap_elements(grid.grabExpr(draw(epiDEGs), wrap = TRUE, wrap.grobs = TRUE)) + ggtitle('Epi') + theme(title = element_text(size = 30))) +
    plot_layout(ncol = 1, heights = c(2, 3, 3, 3, 2)) +
    plot_annotation(tag_levels = 'A') &
    theme(plot.tag = element_text(size = 30))
supp_fig_1

In [ ]:
ggsave(plot = supp_fig_1, 
       filename = 'supp_fig_1.pdf', 
       width = 30, 
       height = 30, 
       units = 'in')

In [ ]:
ggsave(plot = supp_fig_1, 
       filename = 'supp_fig_1.png', 
       width = 30, 
       height = 30, 
       units = 'in')

# umap Strom

In [ ]:
getwd()

In [ ]:
stromG4423 = readr::read_rds('/n/data1/bwh/medicine/korsunsky/lab/mup728/CRC_MERFISH/Fine_typing_with_weighted_KNN/Strom/Strom_fine_types_G4423.rds')

In [ ]:
require(Seurat)

In [ ]:
Embeddings(stromG4423, 'humap') %>% head

In [ ]:
DimPlot(stromG4423 %>% subset(technology == 'scRNA'), reduction = 'humap', raster = TRUE)

In [ ]:
colnames(stromG4423@meta.data)

In [ ]:
options(repr.plot.res = 500, repr.plot.width = 12, repr.plot.height = 12)

DimPlot(stromG4423 %>% subset(technology == 'MERFISH'), reduction = 'humap', raster = TRUE, group.by = 'cleaned_fine_types', label = TRUE) + ggtitle('Strom fine types in G4423')

In [ ]:
FeaturePlot(stromG4423, features = c('CXCL14', 'BMP5', 'TAGLN', 'GREM1', 'INHBA', 'ENHO'))

In [ ]:
geneList = str_split(pattern = '\n', 'MFAP5
SFRP2
CXCL12
CXCL14
CXCR4
BMP4
ENHO
BMP5
BMP2
WNT2B
CCL2
GREM1
INHBA
FAP
PDPN
MMP1
MMP3
CXCL3
CXCL5
CXCL8
CSF3
IL24
RSPO3
TAGLN
ACTA2
MYH11
TNFSF15
CSF1
IL6
NOTCH3
CSPG4
ENHO
TSLP
UBE2C
TOP2A
MKI67
SELP
ACKR1
SELE
NCAM1
SPP1
NRXN1
TCF7
ACTG2
MYH11
GJA4
GJA5
SEMA3G
ESM1
FLT1
CD34
KDR
DLL4
JAG2
VWF
NOTCH4
PDGFB
CD36
PGF
CCL21
PROX1
ACKR2
LYVE1
FLT4') %>% unlist

In [ ]:
dev.off()
p1 = Seurat::DoHeatmap(object = stromG4423 %>% subset(technology == 'MERFISH') %>% SetIdent(value = 'cleaned_fine_types') %>% subset(downsample = 200), features = geneList, group.by = "cleaned_fine_types", slot = 'counts') + scale_fill_viridis_c(direction = -1)
p1

In [ ]:
p1

# Scatterplot of logFC between MyeloidISG and MyeloidISGlow

In [ ]:
require(tidyverse)
require(data.table)
require(ggrepel)
require(cowplot)

In [ ]:
supp_table = fread('supplementary_table_1_fig_1.csv')
head(supp_table)

In [ ]:
unique(supp_table$contrast)

In [ ]:
.temp = supp_table %>%
    filter(contrast %in% c('Tcd8-gdlike', 'Tcd8-gdlike-PD1')) %>%
    select(contrast, feature, logFC) %>%
    pivot_wider(names_from = contrast, values_from = logFC) %>%
    mutate(label = case_when( `Tcd8-gdlike-PD1` > 0.5 |  `Tcd8-gdlike` > 0.5 ~ feature, .default = '')) 
ggplot(.temp, aes(x = `Tcd8-gdlike-PD1`, y = `Tcd8-gdlike`)) +
    geom_point() +
    geom_abline() +
    geom_text_repel(aes(label = label), inherit.aes = TRUE) +
    theme_half_open(10) +
    theme(axis.text = element_text(size = 15), axis.title = element_text(size = 15)) +
    NULL

In [ ]:
.temp = supp_table %>%
    filter(contrast %in% c('Myeloid-ISGlow', 'Myeloid-ISGhigh')) %>%
    select(contrast, feature, logFC) %>%
    pivot_wider(names_from = contrast, values_from = logFC) %>%
    mutate(label = case_when( `Myeloid-ISGlow` > 0.5 |  `Myeloid-ISGhigh` > 0.5 ~ feature, .default = '')) 
ggplot(.temp, aes(x = `Myeloid-ISGlow`, y = `Myeloid-ISGhigh`)) +
    geom_point() +
    geom_abline() +
    geom_text_repel(aes(label = label), inherit.aes = TRUE) +
    theme_half_open(10) +
    theme(axis.text = element_text(size = 15), axis.title = element_text(size = 15)) +
    NULL

In [ ]:
.temp = supp_table %>%
    filter(contrast %in% c('Tcd4-Treg-prolif', 'Tcd4-Treg')) %>%
    select(contrast, feature, logFC) %>%
    pivot_wider(names_from = contrast, values_from = logFC) %>%
    mutate(label = case_when( `Tcd4-Treg-prolif` > 0.5 |  `Tcd4-Treg` > 0.5 ~ feature, .default = '')) 
ggplot(.temp, aes(x = `Tcd4-Treg-prolif`, y = `Tcd4-Treg`)) +
    geom_point() +
    geom_abline() +
    geom_text_repel(aes(label = label), inherit.aes = TRUE) +
    theme_half_open(10) +
    theme(axis.text = element_text(size = 15), axis.title = element_text(size = 15)) +
    NULL

In [ ]:
.temp = supp_table %>%
    filter(contrast %in% c('Tcd8-CXCL13-prolif', 'Tcd8-CXCL13')) %>%
    select(contrast, feature, logFC) %>%
    pivot_wider(names_from = contrast, values_from = logFC) %>%
    mutate(label = case_when( `Tcd8-CXCL13-prolif` > 0.5 |  `Tcd8-CXCL13` > 0.5 ~ feature, .default = '')) 
ggplot(.temp, aes(x = `Tcd8-CXCL13-prolif`, y = `Tcd8-CXCL13`)) +
    geom_point() +
    geom_abline() +
    geom_text_repel(aes(label = label), inherit.aes = TRUE) +
    theme_half_open(10) +
    theme(axis.text = element_text(size = 15), axis.title = element_text(size = 15)) +
    NULL

In [ ]:
.temp = supp_table %>%
    filter(contrast %in% c('Myeloid-DCmreg', 'Myeloid-ISGhigh')) %>%
    select(contrast, feature, logFC) %>%
    pivot_wider(names_from = contrast, values_from = logFC) %>%
    mutate(label = case_when( `Myeloid-DCmreg` > 0.5 |  `Myeloid-ISGhigh` > 0.5 ~ feature, .default = '')) 
ggplot(.temp, aes(x = `Myeloid-DCmreg`, y = `Myeloid-ISGhigh`)) +
    geom_point() +
    geom_abline() +
    geom_text_repel(aes(label = label), inherit.aes = TRUE) +
    theme_half_open(10) +
    theme(axis.text = element_text(size = 15), axis.title = element_text(size = 15)) +
    NULL